In [1]:
# Get pointings
# For each pointing, get the sky loss given:
# a dish model (beam with aperture effects)
# a ionosphere model, and
# a bright source model (D point sources)
# an array

# Sky loss calculation:
# Choose M random l,m within the field of view
# For each frequency:
#   For each bright source:
#     compute model vis
#     accumulate visibilities
#   DFT vis only M directions
#   Compute RMS, with zero-point adjustment +1/(N-1)



In [1]:
import os
import time
from functools import partial

import astropy.constants as const
import astropy.coordinates as ac
import astropy.time as at
import astropy.units as au
import jax.numpy as jnp
import jax.random
import numpy as np

from dsa2000_assets.content_registry import fill_registries
from dsa2000_assets.registries import source_model_registry, array_registry
from dsa2000_common.common.array_types import FloatArray
from dsa2000_common.common.astropy_utils import create_spherical_spiral_grid
from dsa2000_common.common.enu_frame import ENU
from dsa2000_common.common.quantity_utils import time_to_jnp, quantity_to_jnp
from dsa2000_common.common.serialise_utils import SerialisableBaseModel
from dsa2000_common.delay_models.base_far_field_delay_engine import build_far_field_delay_engine, \
    BaseFarFieldDelayEngine
from dsa2000_common.delay_models.base_near_field_delay_engine import build_near_field_delay_engine, \
    BaseNearFieldDelayEngine
from dsa2000_common.delay_models.uvw_utils import perley_lmn_from_icrs
from dsa2000_common.gain_models.beam_gain_model import build_beam_gain_model
from dsa2000_common.gain_models.gain_model import GainModel
from dsa2000_common.geodesics.base_geodesic_model import build_geodesic_model, BaseGeodesicModel
from dsa2000_common.visibility_model.source_models.celestial.base_point_source_model import \
    build_point_source_model_from_wsclean_components, BasePointSourceModel
from dsa2000_fm.systematics.dish_aperture_effects import build_dish_aperture_effects
from dsa2000_fm.systematics.ionosphere import compute_x0_radius, construct_canonical_ionosphere, \
    build_ionosphere_gain_model


@partial(jax.jit)
def compute_sky_values(l, m, n,
                       bright_sky_model: BasePointSourceModel, total_gain_model: GainModel,
                       times: FloatArray, far_field_delay_engine: BaseFarFieldDelayEngine,
                       near_field_delay_engine: BaseNearFieldDelayEngine,
                       geodesic_model: BaseGeodesicModel, freqs: FloatArray,
                       zero_point: FloatArray):
    # For each frequency
    visibilty_coords = far_field_delay_engine.compute_visibility_coords(
        freqs=freqs,
        times=times,
        with_autocorr=False
    )
    vis = bright_sky_model.predict(
        visibility_coords=visibilty_coords,
        gain_model=total_gain_model,
        near_field_delay_engine=near_field_delay_engine,
        far_field_delay_engine=far_field_delay_engine,
        geodesic_model=geodesic_model
    )  # [T, B, C]
    vis = jax.lax.reshape(vis, (vis.shape[0] * vis.shape[1], vis.shape[2]))  # [T*B, C]
    uvw = jax.lax.reshape(visibilty_coords, np.shape(vis)[0:1] + (3,))  # [T*B, 3]

    # DFT
    wavelengths = quantity_to_jnp(const.c) / freqs
    uvw = uvw[:, None, :] / wavelengths[:, None]  # [T*B, C, 3]
    u, v, w = uvw  # [T*B, C]

    def add_square_sum(square_sum, x):
        l, m, n = x
        # 2*pi*freq/c*(l*u + m*v + (n-1)*w)
        delay = l * u + m * v + (n - 1.) * w  # [T*B, C]
        phi = (2 * np.pi) * delay  # [T*B, C]
        fringe = jax.lax.complex(jnp.cos(phi), jnp.sin(phi)).astype(jnp.complex64)  # [T*B, C]
        integrand = (vis * fringe).real.astype(jnp.float32)  # [T*B, C]
        delta = (n * jnp.sum(integrand)).astype(jnp.float32)
        delta /= np.size(vis)
        delta -= zero_point
        square_sum += delta ** 2
        return square_sum.astype(jnp.float32), None

    square_sum, _ = jax.lax.scan(add_square_sum, jnp.array(0., jnp.float32), (l, m, n))
    square_sum /= len(l)
    rms = jnp.sqrt(square_sum)
    return rms


def main(save_folder: str, array_name: str,
         num_measure_points: int,
         angular_radius: au.Quantity,
         bright_source_id: str,
         pointing_offset_stddev: au.Quantity,
         axial_focus_error_stddev: au.Quantity,
         horizon_peak_astigmatism_stddev: au.Quantity
         ):
    key = jax.random.PRNGKey(0)
    fill_registries()
    os.makedirs(save_folder, exist_ok=True)
    array = array_registry.get_instance(array_registry.get_match(array_name))
    times = at.Time("2021-01-01T00:00:00", format="isot", scale="utc")[None]
    ref_time = times[0]
    antennas = array.get_antennas()
    array_location = array.get_array_location()

    phase_center = zenith = ENU(0, 0, 1, obstime=ref_time, location=array_location).transform_to(ac.ICRS())

    freqs = array.get_channels()
    freqs_jax = quantity_to_jnp(freqs)

    far_field_delay_engine = build_far_field_delay_engine(
        antennas=antennas,
        phase_center=phase_center,
        start_time=times.min(),
        end_time=times.max(),
        ref_time=ref_time
    )

    near_field_delay_engine = build_near_field_delay_engine(
        antennas=antennas,
        start_time=times.min(),
        end_time=times.max(),
        ref_time=ref_time
    )

    geodesic_model = build_geodesic_model(
        antennas=antennas,
        array_location=array_location,
        phase_center=phase_center,
        obstimes=times,
        ref_time=ref_time,
        pointings=None
    )

    wsclean_clean_component_file = source_model_registry.get_instance(
        source_model_registry.get_match(bright_source_id)
    ).get_wsclean_clean_component_file()

    bright_sky_model = build_point_source_model_from_wsclean_components(
        wsclean_clean_component_file=wsclean_clean_component_file,
        model_freqs=freqs[[0, -1]],
        full_stokes=False
    )

    directions = ac.ICRS(ra=np.asarray(bright_sky_model.ra) * au.rad,
                         dec=np.asarray(bright_sky_model.dec) * au.rad)

    beam_model = build_beam_gain_model(
        array_name=array_name,
        full_stokes=False,
        times=times,
        ref_time=ref_time,
        freqs=freqs
    )

    dish_aperture_effects = build_dish_aperture_effects(
        dish_diameter=array.get_antenna_diameter(),
        focal_length=array.get_focal_length(),
        elevation_pointing_error_stddev=pointing_offset_stddev,
        cross_elevation_pointing_error_stddev=pointing_offset_stddev,
        axial_focus_error_stddev=axial_focus_error_stddev,
        elevation_feed_offset_stddev=1 * au.mm,  # Assume perpendicular accuracy is quite good.
        cross_elevation_feed_offset_stddev=1 * au.mm,
        horizon_peak_astigmatism_stddev=horizon_peak_astigmatism_stddev,
        # surface_error_mean=0 * au.mm, # TODO: update to use a GP model for RMS surface error
        # surface_error_stddev=1 * au.mm
    )
    key, sample_key = jax.random.split(key)
    beam_model = dish_aperture_effects.apply_dish_aperture_effects(
        sample_key,
        beam_model,
        geodesic_model=geodesic_model
    )

    x0_radius = compute_x0_radius(array_location, ref_time)
    ionosphere = construct_canonical_ionosphere(
        x0_radius=x0_radius,
        turbulent=True,
        dawn=True,
        high_sun_spot=True
    )

    key, sim_key = jax.random.split(key)
    ionosphere_gain_model = build_ionosphere_gain_model(
        key=sim_key,
        ionosphere=ionosphere,
        model_freqs=freqs[[0, len(freqs) // 2, -1]],
        antennas=antennas,
        ref_location=array_location,
        times=times,
        ref_time=ref_time,
        directions=directions,
        phase_centre=phase_center,
        full_stokes=False,
        save_file=os.path.join(save_folder, 'simulated_dtec.json')
    )
    ionosphere_gain_model.plot_regridded_beam(
        save_fig=os.path.join(save_folder, 'regridded_beam.png'),
        ant_idx=-1,
        show=False
    )

    total_gain_model = beam_model @ ionosphere_gain_model

    measure_directions = create_spherical_spiral_grid(
        pointing=phase_center,
        num_points=num_measure_points,
        angular_radius=angular_radius
    )  # [M]

    l, m, n = perley_lmn_from_icrs(measure_directions.ra.rad, measure_directions.dec.rad, phase_center.ra.rad,
                                   phase_center.dec.rad)

    times_jax = time_to_jnp(times, ref_time)

    # For each frequency:
    #   For each bright source:
    #     compute model vis
    #     accumulate visibilities
    #   DFT vis only M directions
    #   Compute RMS, with zero-point adjustment +1/(N-1)
    t0 = time.time()
    rms = jax.block_until_ready(
        compute_sky_values(
            l=l, m=m, n=n,
            bright_sky_model=bright_sky_model,
            total_gain_model=total_gain_model,
            times=times_jax,
            far_field_delay_engine=far_field_delay_engine,
            near_field_delay_engine=near_field_delay_engine,
            geodesic_model=geodesic_model,
            freqs=freqs_jax,
            num_measure_points=num_measure_points
        )
    )
    t1 = time.time()
    print(f"Time taken: {t1 - t0:.2f} s")
    result = Result(
        phase_center=phase_center,
        ref_time=ref_time,
        array_name=array_name,
        bright_source_id=bright_source_id,
        num_measure_points=num_measure_points,
        angular_radius=angular_radius,
        pointing_offset_stddev=pointing_offset_stddev,
        axial_focus_error_stddev=axial_focus_error_stddev,
        horizon_peak_astigmatism_stddev=horizon_peak_astigmatism_stddev,
        rms=rms
    )

    with open(os.path.join(save_folder, 'result.json'), 'w') as f:
        f.write(result.json(indent=2))


class Result(SerialisableBaseModel):
    phase_center: ac.ICRS
    ref_time: at.Time
    array_name: str
    bright_source_id: str
    num_measure_points: int
    angular_radius: au.Quantity
    pointing_offset_stddev: au.Quantity
    axial_focus_error_stddev: au.Quantity
    horizon_peak_astigmatism_stddev: au.Quantity
    rms: np.ndarray


main(
    save_folder='sky_loss_test',
    array_name='dsa2000_optimal_v1',
    num_measure_points=10,
    angular_radius=1.5 * au.deg,
    bright_source_id='mock_calibrators',  #'nvss_calibrators',
    pointing_offset_stddev=1 * au.arcmin,
    axial_focus_error_stddev=1 * au.mm,
    horizon_peak_astigmatism_stddev=1 * au.mm
)







/home/albert/git/DSA2000-Cal/dsa2000_cal/src/dsa2000_common/common/mixed_precision_utils.py:10: UserWarning: JAX x64 is not enabled. Setting it now, but check for errors.
  warnings.warn("JAX x64 is not enabled. Setting it now, but check for errors.")


Searching for sync certificate: /home/albert/git/DSA2000-Cal/dsa2000_cal/src/dsa2000_assets/arrays/dsa2000_optimal_v1/.sync_cert
Searching for sync certificate: /home/albert/git/DSA2000-Cal/dsa2000_cal/src/dsa2000_assets/source_models/calibrators/.sync_cert
[]


IndexError: list index out of range